<a href="https://colab.research.google.com/github/angelinawong1210/VietnameseFakeNews/blob/main/FakeNews_SVM_Gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
truenews = pd.read_csv('/content/Fake.csv')
fakenews = pd.read_csv('/content/Real.csv')

In [ ]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [ ]:
# Combine the 2 DataFrames into a single data frame
news = pd.concat([truenews, fakenews])
news["Article"] = news["tittle"] + news["text"]
news.sample(frac = 1) #Shuffle 100%

,tittle,text,domain,topic,True/Fake,Article
111,"Thấy vợ nằm ""thoát y"" bên cạnh ông bạn thân, n...",Vợ không mặc quần áo nằm bên cạnh người đàn ôn...,NaN,Đời sống,True,"Thấy vợ nằm ""thoát y"" bên cạnh ông bạn thân, n..."
91,Nghi phạm sát hại 3 người trong gia đình ở Tiề...,3 người trong gia đình ở Tiền Giang bị sát hạ...,vnexpress.net,An ninh,Fake,Nghi phạm sát hại 3 người trong gia đình ở Tiề...
61,Cuộc sống ở 'ốc đảo' biệt thự ngoại thành Hà Nội,Loanh quanh tập thể dục một mình mãi cũng chá...,doisong.vnexpress.net,Đời sống,Fake,Cuộc sống ở 'ốc đảo' biệt thự ngoại thành Hà N...
107,Trí tuệ cổ nhân: Vương triều diệt vong ứng với...,Cổ nhân cho rằng giữa Trời và người là có thể ...,trithucvn.org,Đời sống,True,Trí tuệ cổ nhân: Vương triều diệt vong ứng với...
147,"Cô gái bị thanh niên đi SH giật dây chuyền, tà...","B͏an đầu tưởng đôi tình nhân đang cãi nhau, nh...",NaN,An ninh,True,"Cô gái bị thanh niên đi SH giật dây chuyền, tà..."
...,...,...,...,...,...,...
65,Người Hà Nội chen chân chụp ảnh trong vườn cúc...,"12h trưa ngày cuối tuần, nhiều người không ng...",vnexpress.net,Đời sống,Fake,Người Hà Nội chen chân chụp ảnh trong vườn cúc...
126,Hải Phòng: Xúc động khi Bố con người đàn ông “...,A͏n͏h͏ Η͏à v͏à ᴄ͏o͏n͏ t͏r͏a͏ι͏ ƌ͏ã ƌ͏ư͏ợᴄ͏ ở ...,NaN,Đời sống,True,Hải Phòng: Xúc động khi Bố con người đàn ông “...
66,Hot girl có vòng 1 tự nhiên khủng nhất châu Á ...,Loạt ảnh mới nhất về cô gái có vòng 1 khủng n...,tinvn.info,Đời sống,True,Hot girl có vòng 1 tự nhiên khủng nhất châu Á ...
126,Nga nối lại thỏa thuận ngũ cốc Ukraine,Nga xác nhận tiếp tục tham gia thỏa thuận ngũ ...,vnexpress.net,Chính trị,Fake,Nga nối lại thỏa thuận ngũ cốc UkraineNga xác ...


In [ ]:
import nltk 
nltk.download('stopwords')
#Data Cleaning
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def process_text(s):

    # Check string to see if they are a punctuation
    nopunc = [char for char in s if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in nopunc.split()]
    return clean_string

In [ ]:
# Tokenize the Article
#rerun, takes LOOOONG
news['Clean Text'] = news['Article'].apply(process_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_)) #Total vocab words


298


In [ ]:
#Bag-of-Words (bow) transform the entire DataFrame of text
news_bow = bow_transformer.transform(news['Clean Text'])

In [ ]:
print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

Shape of Sparse Matrix:  (299, 298)
Amount of Non-Zero occurences:  299


In [ ]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(299, 298)


In [ ]:
#Train SVM model
from sklearn.svm import SVC
fakenews_detect_model = SVC(kernel = 'linear', random_state = 0).fit(news_tfidf, news['True/Fake'])


In [ ]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))


              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       150
        True       1.00      1.00      1.00       149

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299



In [ ]:
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

209 90 299


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', SVC(kernel = 'linear', random_state = 0)),  
])
pipeline.fit(news_train,text_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x7fd27c948310>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', SVC(kernel='linear', random_state=0))])

In [ ]:
prediction = pipeline.predict(news_test)

In [ ]:
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.86      0.86      0.86        44
        True       0.87      0.87      0.87        46

    accuracy                           0.87        90
   macro avg       0.87      0.87      0.87        90
weighted avg       0.87      0.87      0.87        90

